# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.9384918657156156),
 (1, 2.9384918657156156),
 (2, 2.9384918657156156),
 (3, 2.9384918657156156),
 (4, 2.9384918657156156)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('it', 18)]),
 (1, [('banana', 2), ('is', 18), ('what', 10)])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.007435339455865164),
   (None, 0.02651165321186033),
   (None, 0.08685660088271852),
   (None, 0.1647210964530097),
   (None, 0.1919425374830409),
   (None, 0.1994892798595106),
   (None, 0.21895693703011176),
   (None, 0.22789553049321132),
   (None, 0.25516778645330396),
   (None, 0.25754177978471626),
   (None, 0.25761201931660604),
   (None, 0.26610200927064265),
   (None, 0.2804514767733065),
   (None, 0.2875844274898425),
   (None, 0.34358953934561054),
   (None, 0.4251476136834764),
   (None, 0.43067826746495186),
   (None, 0.45463240498727575),
   (None, 0.4551076984436122),
   (None, 0.4814882189862506)]),
 (1,
  [(None, 0.5418394078091472),
   (None, 0.550785619011714),
   (None, 0.6153680955432717),
   (None, 0.6811523777926556),
   (None, 0.6853083302729692),
   (None, 0.6898206790330181),
   (None, 0.745951019451846),
   (None, 0.8904613288171936),
   (None, 0.9732111949630787),
   (None, 0.9882439747917753)])]

In [ ]:
# выборки данных из папки data для проверки (stations.csv, stations2.csv, stations_intersection)
from google.colab import files

uploaded = files.upload()

Saving stations_intersection.csv to stations_intersection.csv


# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [ ]:
import csv
from typing import NamedTuple, Iterator, List
from collections import defaultdict

class KeyValue(NamedTuple):
  key: str
  val: float

def RECORDREADER(file: str) -> Iterator[KeyValue]:
  with open(file, mode='r') as file:
    r = csv.reader(file)
    next(r)
    for row in r:
      val = float(row[2]) # беру 3 столбец
      yield KeyValue(key=None, val=val)

def MAP(key: str, val: float) -> Iterator[KeyValue]:
  yield KeyValue(key=None, val=val)

def REDUCE(key: str, vals: Iterator[float]) -> Iterator[KeyValue]:
  yield KeyValue(key=None, val=max(vals)) # находим макс. значение

def MapReduce(RECORDREADER, MAP, REDUCE, file: str):
  r_data = list(RECORDREADER(file)) # чтение из csv-файла

  map_data = [] # MAP для каждого элемента
  for kv in r_data:
    map_data.extend(MAP(kv.key, kv.val))

  group_data = {} # группировка данных по ключу (none)
  for kv in map_data:
    group_data[kv.key] = group_data.get(kv.key, []) + [kv.val]

  reduce_data = [] # REDUCE для сгрупированных данных
  for key, vals in group_data.items():
    reduce_data.extend(REDUCE(key, iter(vals)))

  return reduce_data


res = MapReduce(RECORDREADER, MAP, REDUCE, 'stations.csv') # запуск MapReduce для обработки
print(f"Max value lat: {res[0].val}")

Max value lat: 37.80477


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [ ]:
def MAP(key: str, val: float) -> Iterator[KeyValue]:
  yield KeyValue(key=None, val=val)

def REDUCE(key: str, vals: Iterator[float]) -> Iterator[KeyValue]:
  total_sum = 0
  count = 0
  for val in vals: # подсчет суммы и кол-ва элементов
    total_sum += val
    count += 1
  if count > 0: # нахождение среднего значения
    mean = total_sum / count
    yield KeyValue(key=None, val=mean)
  else:
    yield KeyValue(key=None, val=0)


res = MapReduce(RECORDREADER, MAP, REDUCE, 'stations.csv')
print(f"Arithmetic mean lat: {res[0].val}")

Arithmetic mean lat: 37.59024338428572


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
class KeyValue(NamedTuple):
  key: int
  val: str

def RECORDREADER(file: str) -> Iterator[KeyValue]:
  with open(file, mode='r') as file:
    r = csv.reader(file)
    next(r)
    for row in r:
      key = int(row[4])  # ключ - 5 столбец
      val = row[5]  # значение - 6 столбец
      yield KeyValue(key=key, val=val)

def MAP(key: int, val: str) -> Iterator[KeyValue]:
  yield KeyValue(key=key, val=val) # передача без изменений

def groupbykey(sort_data: List[KeyValue]) -> List[KeyValue]:
  group = []
  curr_key = None
  curr_vals = []

  for kv in sort_data:
    if kv.key == curr_key:
      curr_vals.append(kv.val) # при совпадении ключа с предыдущ., значение добавляется в группу
    else:
      if curr_key is not None:
        group.append(KeyValue(curr_key, curr_vals)) # при изменении ключа, предыдущ. группа сохраняется
      curr_key = kv.key # новая группа для нового ключа
      curr_vals = [kv.val]

  if curr_key is not None: # добавление последней группы
    group.append(KeyValue(curr_key, curr_vals))

  return group

def MapReduce(RECORDREADER, MAP, groupbykey, file: str):
  r_data = list(RECORDREADER(file)) # чтение данных

  map_data = [] # MAP для каждого элемента
  for kv in r_data:
    map_data.extend(MAP(kv.key, kv.val))

  sort_data = sorted(map_data, key=lambda kv: kv.key) # сортировка данных по ключу
  group_data = groupbykey(sort_data) # группировка значений по ключу

  return group_data


res = MapReduce(RECORDREADER, MAP, groupbykey, 'stations.csv')
for group in res:
  print(f"Key: {group.key}, Values: {group.val}")

Key: 11, Values: ['San Jose', 'Mountain View', 'Palo Alto', 'Palo Alto']
Key: 15, Values: ['San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'Redwood City', 'Redwood City', 'Redwood City', 'Redwood City', 'Redwood City', 'Mountain View', 'Mountain View', 'Mountain View', 'Mountain View', 'Palo Alto', 'Palo Alto', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Jose', 'San Francisco', 'Redwood City', 'San Jose']
Key: 19, Values: ['San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco']
Key: 23, Values: ['Mountain View', 'Mountain View', 'Palo Alto', 'San Francisco', 'San Fran

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
class KeyValue(NamedTuple):
  key: str
  val: str

def RECORDREADER(file: str) -> Iterator[KeyValue]:
  with open(file, mode='r') as file:
    r = csv.reader(file)
    next(r)
    for row in r:
      yield KeyValue(key=None, val=row[5]) # беру 6 столбец

def MAP(key: str, val: str) -> Iterator[KeyValue]:
  yield KeyValue(key=None, val=val) # данные без изменений

def REDUCE(key: str, vals: Iterator[str]) -> Iterator[KeyValue]:
  unique_vals = set(vals)  # исключаем дубликаты
  for val in unique_vals: # оставляем только уникальные значения
    yield KeyValue(key=None, val=val)

def MapReduce(RECORDREADER, MAP, REDUCE, file: str):
  r_data = list(RECORDREADER(file))

  map_data = []
  for kv in r_data:
    map_data.extend(MAP(kv.key, kv.val))

  group_data = {} # группировка данных по ключу (none)
  for kv in map_data:
    group_data[kv.key] = group_data.get(kv.key, []) + [kv.val]

  reduce_data = [] # применение REDUCE
  for key, vals in group_data.items():
    reduce_data.extend(REDUCE(key, iter(vals)))

  return reduce_data


res = MapReduce(RECORDREADER, MAP, REDUCE, 'stations.csv')
for kv in res:
  print(f"Unique Value: {kv.val}")

Unique Value: Mountain View
Unique Value: San Jose
Unique Value: Redwood City
Unique Value: Palo Alto
Unique Value: San Francisco


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
class KeyValue(NamedTuple):
  key: str
  val: str

def RECORDREADER(file: str) -> Iterator[KeyValue]:
  with open(file, mode='r') as file:
    r = csv.reader(file)
    next(r)
    for row in r:
      yield KeyValue(key=None, val=row[0]) # беру 1 столбец

def MAP(key: str, val: str, predicate) -> Iterator[KeyValue]:
  if predicate(val): # проверка на выполнение условий
    yield KeyValue(key=val, val=val) # пара (t, t)

def REDUCE(key: str, vals: Iterator[str]) -> Iterator[KeyValue]:
  for val in vals: # вернем отфильтрованные значения
    yield KeyValue(key=val, val=val)

def predicate(val: str) -> bool:
  try:
    return float(val) > 65 # проверка значения на > 65
  except ValueError:
    return False

def MapReduce(RECORDREADER, MAP, REDUCE, file: str, predicate):
  r_data = list(RECORDREADER(file))

  map_data = []
  for kv in r_data:
    map_data.extend(MAP(kv.key, kv.val, predicate))

  group_data = {}
  for kv in map_data:
    group_data[kv.key] = group_data.get(kv.key, []) + [kv.val]

  reduce_data = []
  for key, vals in group_data.items():
    reduce_data.extend(REDUCE(key, iter(vals)))

  return reduce_data


res = MapReduce(RECORDREADER, MAP, REDUCE, 'stations.csv', predicate)
for kv in res:
  print(f"Selection: {kv.val}")

Selection: 66
Selection: 67
Selection: 68
Selection: 69
Selection: 70
Selection: 71
Selection: 72
Selection: 73
Selection: 74
Selection: 75
Selection: 76
Selection: 77
Selection: 80
Selection: 82
Selection: 83
Selection: 84


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [ ]:
class KeyValue(NamedTuple):
  key: str
  val: str

def RECORDREADER(file: str) -> Iterator[KeyValue]:
  with open(file, mode='r') as file:
    r = csv.reader(file)
    next(r)
    for row in r:
      yield KeyValue(key=None, val=row) # в этот раз берем весь ряд как значение

def MAP(key: str, val: List[str], attrib: List[int]) -> Iterator[KeyValue]:
  temp_tuple = tuple(val[i] for i in attrib) # проекция на множество атрибутов S
  yield KeyValue(key=str(temp_tuple), val=str(temp_tuple))  # пара (t', t')

def REDUCE(key: str, vals: Iterator[str]) -> Iterator[KeyValue]:
  for val in vals: # только уникальные проекции
    yield KeyValue(key=val, val=val)

def MapReduce(RECORDREADER, MAP, REDUCE, file: str, attrib: List[int]):
  r_data = list(RECORDREADER(file))

  map_data = []
  for kv in r_data:
    map_data.extend(MAP(kv.key, kv.val, attrib))

  group_data = {}
  for kv in map_data:
    group_data[kv.key] = group_data.get(kv.key, []) + [kv.val]

  reduce_data = []
  for key, vals in group_data.items():
    reduce_data.extend(REDUCE(key, iter(vals)))

  return reduce_data


attrib = [0, 5] # мн-во атрибутов для проекции
res = MapReduce(RECORDREADER, MAP, REDUCE, 'stations.csv', attrib)
for kv in res:
  print(f"Projection: {kv.val}")

Projection: ('2', 'San Jose')
Projection: ('3', 'San Jose')
Projection: ('4', 'San Jose')
Projection: ('5', 'San Jose')
Projection: ('6', 'San Jose')
Projection: ('7', 'San Jose')
Projection: ('8', 'San Jose')
Projection: ('9', 'San Jose')
Projection: ('10', 'San Jose')
Projection: ('11', 'San Jose')
Projection: ('12', 'San Jose')
Projection: ('13', 'San Jose')
Projection: ('14', 'San Jose')
Projection: ('16', 'San Jose')
Projection: ('21', 'Redwood City')
Projection: ('22', 'Redwood City')
Projection: ('23', 'Redwood City')
Projection: ('24', 'Redwood City')
Projection: ('25', 'Redwood City')
Projection: ('26', 'Redwood City')
Projection: ('27', 'Mountain View')
Projection: ('28', 'Mountain View')
Projection: ('29', 'Mountain View')
Projection: ('30', 'Mountain View')
Projection: ('31', 'Mountain View')
Projection: ('32', 'Mountain View')
Projection: ('33', 'Mountain View')
Projection: ('34', 'Palo Alto')
Projection: ('35', 'Palo Alto')
Projection: ('36', 'Palo Alto')
Projection: ('37

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [ ]:
class KeyValue(NamedTuple):
  key: str
  val: str

def RECORDREADER(file: str) -> Iterator[KeyValue]:
  with open(file, mode='r') as file:
    r = csv.reader(file)
    next(r)
    for row in r:
      yield KeyValue(key=None, val=row) # снова же берем ряд

def MAP(key: str, val: List[str]) -> Iterator[KeyValue]:
  yield KeyValue(key=str(val), val=str(val)) # создание пары (t, t)

def REDUCE(key: str, vals: Iterator[str]) -> Iterator[KeyValue]:
  for val in vals: # объединенная запись (создание одной пары)
    yield KeyValue(key=val, val=val)

def MapReduce(RECORDREADER, MAP, REDUCE, files: List[str]):
  r_data = [] # берем данные из всех файлов
  for file in files:
    r_data.extend(list(RECORDREADER(file)))

  map_data = []
  for kv in r_data:
    map_data.extend(MAP(kv.key, kv.val))

  reduce_data = []
  reduce_data.extend(REDUCE(None, iter([kv.val for kv in map_data])))

  return reduce_data


res = MapReduce(RECORDREADER, MAP, REDUCE, ['stations.csv', 'stations2.csv'])
for kv in res:
  print(f"Union: {kv.val}")

Union: ['2', 'San Jose Diridon Caltrain Station', '37.329732', '-121.90178200000001', '27', 'San Jose', '8/6/2013']
Union: ['3', 'San Jose Civic Center', '37.330698', '-121.888979', '15', 'San Jose', '8/5/2013']
Union: ['4', 'Santa Clara at Almaden', '37.333988', '-121.894902', '11', 'San Jose', '8/6/2013']
Union: ['5', 'Adobe on Almaden', '37.331415', '-121.8932', '19', 'San Jose', '8/5/2013']
Union: ['6', 'San Pedro Square', '37.336721000000004', '-121.894074', '15', 'San Jose', '8/7/2013']
Union: ['7', 'Paseo de San Antonio', '37.333798', '-121.88694299999999', '15', 'San Jose', '8/7/2013']
Union: ['8', 'San Salvador at 1st', '37.330165', '-121.88583100000001', '15', 'San Jose', '8/5/2013']
Union: ['9', 'Japantown', '37.348742', '-121.89471499999999', '15', 'San Jose', '8/5/2013']
Union: ['10', 'San Jose City Hall', '37.337391', '-121.886995', '15', 'San Jose', '8/6/2013']
Union: ['11', 'MLK Library', '37.335885', '-121.88566000000002', '19', 'San Jose', '8/6/2013']
Union: ['12', 'S

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [ ]:
class KeyValue(NamedTuple):
  key: str
  val: str

def MAP(val: List[str]) -> Iterator[KeyValue]:
  yield KeyValue(key=str(val), val=str(val)) # пара (t, t)

def REDUCE(vals: Iterator[str]) -> Iterator[KeyValue]:
  val_list = list(vals) # все значения в список
  if len(val_list) == 2 and val_list[0] == val_list[1]: # если два ключа это два одинак. элемента
    yield KeyValue(key=val_list[0], val=val_list[1]) # создаем пару (t, t)

def MapReduce(data_A: List[List[str]], data_B: List[List[str]], MAP, REDUCE):
  map_data_A = [kv for val in data_A for kv in MAP(val)] # MAP для обоих списков
  map_data_B = [kv for val in data_B for kv in MAP(val)]
  total_map_data = map_data_A + map_data_B # объединение данных

  group_data = defaultdict(list) # группировка всех элементов по ключу
  for kv in total_map_data:
    group_data[kv.key].append(kv.val)

  reduce_data = []
  for key, vals in group_data.items():
    reduce_data.extend(REDUCE(iter(vals)))

  return reduce_data


data_A = [('Albina', '6405', 'A'), ('Petr', '6402', 'P'), ('Maria', '6401', 'M')]
data_B = [('Albina', '6405', 'A'), ('Roman', '6405', 'R')]

res = MapReduce(data_A, data_B, MAP, REDUCE)
for kv in res:
  print(f"Intersection: {kv.val}")

Intersection: ('Albina', '6405', 'A')


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [ ]:
class KeyValue(NamedTuple):
  key: tuple
  val: str

def MAP(val: List[str], relation: str) -> Iterator[KeyValue]:
  yield KeyValue(key=tuple(val), val=relation) # сопоставление знач. с его отношением R, S

def REDUCE(vals: Iterator[str], key: tuple) -> Iterator[KeyValue]:
  val_list = list(vals)
  if val_list == ['R']: # если элемент только в R
    yield KeyValue(key=key, val=key) # пара (t, t)

def MapReduce(data_R: List[List[str]], data_S: List[List[str]], MAP, REDUCE):
  map_data_R = [kv for val in data_R for kv in MAP(val, 'R')]
  map_data_S = [kv for val in data_S for kv in MAP(val, 'S')]
  total_map_data = map_data_R + map_data_S

  group_data = defaultdict(list)
  for kv in total_map_data:
    group_data[kv.key].append(kv.val)

  reduce_data = []
  for key, vals in group_data.items():
    reduce_data.extend(REDUCE(iter(vals), key))

  return reduce_data


data_R = [('Total', '120', 'A'), ('Sum', '240', 'B'), ('Max', '43', 'C')] # мн-ва R, S
data_S = [('Max', '43', 'C'), ('Diff', '48', 'A')]

res = MapReduce(data_R, data_S, MAP, REDUCE)
for kv in res:
  print(f"Difference: {kv.val}")

Difference: ('Total', '120', 'A')
Difference: ('Sum', '240', 'B')


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
class KeyValue(NamedTuple):
  key: tuple
  val: tuple

def MAP_R(val: List[str]) -> Iterator[KeyValue]:
  b = val[1]
  a = val[0]
  yield KeyValue(key=(b,), val=('R', a)) # преобр. (a, b) из R в пару (b, (R, a))

def MAP_S(val: List[str]) -> Iterator[KeyValue]:
  b = val[0]
  c = val[1]
  yield KeyValue(key=(b,), val=('S', c)) # преобр. (b, c) из S в пару (b, (S, c))

def REDUCE(vals: Iterator[tuple], key: tuple) -> Iterator[KeyValue]:
  vals_R = []  # значения из R
  vals_S = []  # значения из S

  for val_type, val in vals: # сортировка значений (R, S)
    if val_type == 'R':
        vals_R.append(val)
    elif val_type == 'S':
        vals_S.append(val)

  for a in vals_R: # для каждого (R, a) и (S, c)
    for c in vals_S:
      yield KeyValue(key=key, val=(a, key[0], c)) # создается (a, b, c)

def MapReduce(data_R: List[List[str]], data_S: List[List[str]], MAP_R, MAP_S, REDUCE):
  map_data_R = [kv for val in data_R for kv in MAP_R(val)]
  map_data_S = [kv for val in data_S for kv in MAP_S(val)]
  total_map_data = map_data_R + map_data_S

  group_data = defaultdict(list) # группировка элементов по ключу b
  for kv in total_map_data:
    group_data[kv.key].append(kv.val)

  reduce_data = []
  for key, vals in group_data.items():
    reduce_data.extend(REDUCE(iter(vals), key))

  return reduce_data


data_R = [('Patient_1', 'Blood type: A'),
          ('Patient_2', 'Blood type: B'),
          ('Patient_3', 'Blood type: AB')]
data_S = [('Blood type: A', 'Good'),
          ('Blood type: B', 'Good'),
          ('Blood type: AB', 'Bad')]

res = MapReduce(data_R, data_S, MAP_R, MAP_S, REDUCE)
for kv in res:
  print(f"Natural Join: {kv.val}")

Natural Join: ('Patient_1', 'Blood type: A', 'Good')
Natural Join: ('Patient_2', 'Blood type: B', 'Good')
Natural Join: ('Patient_3', 'Blood type: AB', 'Bad')


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
class KeyValue(NamedTuple):
  key: tuple
  val: int

def MAP(val: List[str]) -> Iterator[KeyValue]:
  a = val[0] # первый элемент - ключ
  b = int(val[1]) # второй элемент - значение
  yield KeyValue(key=(a,), val=b) # из (a, b, c) в (a, b)

def REDUCE(vals: Iterator[int], key: tuple, aggregation_type: str) -> Iterator[KeyValue]:
  val_list = list(vals)

  if aggregation_type == "SUM": # тип агрегации (сумма или макс. значение)
    res = sum(val_list)
  elif aggregation_type == "MAX":
    res = max(val_list)

  yield KeyValue(key=key, val=res) # результат агрегации

def MapReduce(data: List[List[str]], MAP, REDUCE, aggregation_type: str):
  map_data = [kv for val in data for kv in MAP(val)]

  group_data = defaultdict(list) # группировка данных по ключу a
  for kv in map_data:
    group_data[kv.key].append(kv.val)

  reduce_data = [] # REDUCE для каждой группы
  for key, vals in group_data.items():
    reduce_data.extend(REDUCE(iter(vals), key, aggregation_type))

  return reduce_data


data = [('Age', '28', 'A'),               # (a, b, c)
        ('Amount of product', '120', 'C'),
        ('Age', '6', 'B'),
        ('Degrees', '20', 'B'),
        ('Amount of product', '380', 'C')]

res_SUM = MapReduce(data, MAP, REDUCE, "SUM")
res_MAX = MapReduce(data, MAP, REDUCE, "MAX")
print("Aggregation (SUM):")
for kv in res_SUM:
  print(f"{kv.key}: {kv.val}")
print("\nAggregation (MAX):")
for kv in res_MAX:
  print(f"{kv.key}: {kv.val}")

Aggregation (SUM):
('Age',): 34
('Amount of product',): 500
('Degrees',): 20

Aggregation (MAX):
('Age',): 28
('Amount of product',): 380
('Degrees',): 20


#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [ ]:
class KeyValue(NamedTuple):
  key: int # индекс строки матрицы
  val: float

def MAP(row: List[int], vect: List[int], row_ind: int) -> List[KeyValue]:
  # создание пары для каждой строки (индекс, результат умножения)
  return [KeyValue(key=row_ind, val=row[j] * vect[j]) for j in range(len(vect))]

def REDUCE(vals: List[float], key: int) -> float:
  return sum(vals) # сумма всех значений в строке

def split_vect(vect: List[int], portion: int) -> List[List[int]]:
  # разбиваем вектор по порциям
  return [vect[i:i + portion] for i in range(0, len(vect), portion)]

def MapReduce(matr: List[List[int]], vect: List[int], MAP, REDUCE):
  # проверка на размер и его деление
  vect_portions = split_vect(vect, 2) if len(vect) > 3 else [vect]

  map_data = [] # MAP для каждой строки матрицы
  for row_ind, row in enumerate(matr):
    for vect_portion in vect_portions:
      map_data.extend(map(lambda r: MAP(row, vect_portion, row_ind), [row]))

  group_data = defaultdict(list) # группировка данных по индексу строки
  for kv in map_data:
    for item in kv:
      group_data[item.key].append(item.val)

  reduce_data = [(key, REDUCE(vals, key)) for key, vals in group_data.items()]

  return reduce_data


matr = [[4, 0, 2],  # матрица A
        [8, 3, 2],
        [3, 5, 6]]
vect = [2, 1, 2]    # вектор b

res = MapReduce(matr, vect, MAP, REDUCE)
print("Matrix-Vector Multiplication:")
# 4 * 2 + 0 * 1 + 2 * 2 = 12
# 8 * 2 + 3 * 1 + 2 * 2 = 23
# 3 * 2 + 5 * 1 + 6 * 2 = 23
for key, val in res:
  print(f"Row {key}: {val}")

Matrix-Vector Multiplication:
Row 0: 12
Row 1: 23
Row 2: 23


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [ ]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  # solution code that yield(k2,v2) pairs

def REDUCE(key, values):
  (i, k) = key
  # solution code that yield(k3,v3) pairs

Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
import numpy as np
from collections import defaultdict
import random

# работаем с большой матрицей, элементы переходят в ((j, k), big_matr[j, k])
def RECORDREADER():
  for j in range(big_matr.shape[0]): # строки
    for k in range(big_matr.shape[1]): # столбцы
      yield ((j, k), big_matr[j, k])

def MAP(k1, v1):
  (j, k) = k1 # разбивка ключа
  w = v1 # значение большой матрицы
  # перебор строк маленькой матрицы и создание пар
  for i in range(small_matr.shape[0]):
    # ключ - (i, k), значение - small_mat[i, j] * big_mat[j, k]
    yield ((i, k), small_matr[i, j] * w)

def REDUCE(key, vals):
  yield (key, sum(vals)) # для каждого ключа (i, k) ищется сумма произведений

def MapReduce(RECORDREADER, MAP, REDUCE):
  r_datas = list(RECORDREADER())

  map_data = []
  for r_data in r_datas:
    map_data.extend(MAP(r_data[0], r_data[1]))

  group_data = defaultdict(list) # группировка по ключу (i, k)
  for k, v in map_data:
    group_data[k].append(v)

  reduce_data = [] # REDUCE для каждого ключа
  for key, vals in group_data.items():
    reduce_data.extend(REDUCE(key, vals))

  return reduce_data

# преобразование результата в матрицу
def res_for_matr(reduce_out):
  reduce_output = list(reduce_out)
  I = max(i for ((i, k), vw) in reduce_out) + 1 # кол-во строк
  K = max(k for ((i, k), vw) in reduce_out) + 1 # кол-во столбцов
  matr = np.empty(shape=(I, K)) # пустая матрица
  for ((i, k), vw) in reduce_out: # ее заполнение
    matr[i, k] = vw
  return matr


I = 2
J = 3
K = 4 * 10
small_matr = np.random.rand(I, J)
big_matr = np.random.rand(J, K)

numpy_solution = np.matmul(small_matr, big_matr)
my_solution = MapReduce(RECORDREADER, MAP, REDUCE)

np.allclose(numpy_solution, res_for_matr(my_solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [ ]:
I, J, K = 2, 3, 4
small_matr = np.random.rand(I, J)
big_matr = np.random.rand(J, K)

# чтение элементов малой матрицы
def RECORDREADER_S():
  for i in range(I):
    for j in range(J):
      yield (("S", i, j), small_matr[i, j]) # указание, что small

# чтение элементов большой матрицы
def RECORDREADER_B():
  for j in range(J):
    for k in range(K):
      yield (("B", j, k), big_matr[j, k]) # указание, что big

# объединение данных из матриц в один список
def INPUTFORMAT():
  return list(RECORDREADER_S()) + list(RECORDREADER_B())

def MAP(key, val):
  type_matr, a, b = key # тип и индексы

  if type_matr == "S": # если элемент относится к малой матрице
    i, j = a, b
    for k in range(K): # бежим по k
      yield ((i, k), ('S', j, val)) # передача (i, k) и значения

  elif type_matr == "B": # если элемент относится к большой матрице
    j, k = a, b
    for i in range(I): # бежим по i
      yield ((i, k), ('B', j, val))

def COMBINER(key, vals):
  group_S = defaultdict(list) # группировка значений из матриц
  group_B = defaultdict(list)

  for type_matr, j, val in vals: # разделение значений по типу матрицы
    if type_matr == 'S':
      group_S[j].append(val)
    elif type_matr == 'B':
      group_B[j].append(val)

  # вычисление произведения элементов S и B, если имеют общий индекс j
  res = sum(v_m * v_n for j in group_S if j in group_B for v_m in group_S[j] for v_n in group_B[j])

  yield key, res

def REDUCE(key, vals):
  yield key, sum(vals) # сложение всех полученных результатов из (i, k)

def MapReduce(data, MAP, REDUCE, COMBINER=None):
  temp = defaultdict(list)

  for rec in data:
    for key, val in MAP(*rec):
      temp[key].append(val)

  # COMBINER для частичного уменьшения объема
  if COMBINER:
    combine = defaultdict(list)
    for key, vals in temp.items():
      for c_key, c_val in COMBINER(key, vals):
        combine[c_key].append(c_val)
    temp = combine # обновление данных

  res = {}
  for key, vals in temp.items():
    for r_key, r_val in REDUCE(key, vals):
      res[r_key] = r_val # запись конечного результата

  return res


data = INPUTFORMAT() # получение входных данных
reduce_data = MapReduce(data, MAP, REDUCE, COMBINER)

# функция восстановления матрицы
def recovery_matr(reduce_data, I, K):
  matr = np.zeros((I, K)) # пустая матрица
  for (i, k), val in reduce_data.items():
    matr[i, k] = val # заполнение значениями
  return matr

ref = np.matmul(small_matr, big_matr) # пользуемся numpy
solution = recovery_matr(reduce_data, I, K) # восстанавливаем матрицу

np.allclose(solution, ref)

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [ ]:
I, J, K = 2, 3, 4
small_matr = np.random.rand(I, J)
big_matr = np.random.rand(J, K)

# чтение малой матрицы
def RECORDREADER_S_NUM1():
  for i in range(I):
    for j in range(J // 2): # первая половина столбцов
      yield ("S", i, j), small_matr[i, j]

def RECORDREADER_S_NUM2():
  for i in range(I):
    for j in range(J // 2, J): # вторая половина
      yield ("S", i, j), small_matr[i, j]

# чтение большой матрицы
def RECORDREADER_B_NUM1():
  for j in range(J):
    for k in range(K // 2): # первая половина столбцов
      yield ("B", j, k), big_matr[j, k]

def RECORDREADER_B_NUM2():
  for j in range(J):
    for k in range(K // 2, K): # вторая половина
      yield ("B", j, k), big_matr[j, k]

# объединение данных
def INPUTFORMAT():
  return list(RECORDREADER_S_NUM1()) + list(RECORDREADER_S_NUM2()) + list(RECORDREADER_B_NUM1()) + list(RECORDREADER_B_NUM2())

# разбиение данных на пары
def MAP(key, val):
  type_matr, a, b = key

  if type_matr == "S": # если элемент из малой матрицы
    i, j = a, b
    for k in range(K): # бежим по k
      yield ((i, k), ('S', j, val)) # создание ключа (i, k) и значение

  elif type_matr == "B": # если элемент из большой матрицы
    j, k = a, b
    for i in range(I): # бежим по i
      yield ((i, k), ('B', j, val))

def COMBINER(key, vals):
  group_S = defaultdict(list) # группировка значений из матриц
  group_B = defaultdict(list)

  for type_matr, j, val in vals: # деление данных по типу матрицы
    if type_matr == 'S':
      group_S[j].append(val) # значение из малой матрицы
    elif type_matr == 'B':
      group_B[j].append(val) # значение из большой матрицы

  # произведение элементов с одинак. индексом j
  res = sum(v_m * v_n for j in group_S if j in group_B for v_m in group_S[j] for v_n in group_B[j])

  yield key, res

def REDUCE(key, vals):
  yield key, sum(vals) # сложение всех полученных сумм

def MapReduce(data, MAP, REDUCE, COMBINER=None):
  temp = defaultdict(list)

  for rec in data: # создание пар ключ, значение
    for key, val in MAP(*rec):
      temp[key].append(val)

  # частичное вычисление
  if COMBINER:
    combine = defaultdict(list)
    for key, vals in temp.items():
      for c_key, c_val in COMBINER(key, vals):
        combine[c_key].append(c_val)
    temp = combine

  # REDUCE для получения конечного результата
  res = {}
  for key, vals in temp.items():
    for r_key, r_val in REDUCE(key, vals):
      res[r_key] = r_val

  return res


data = INPUTFORMAT() # входные данные
reduce_data = MapReduce(data, MAP, REDUCE, COMBINER)

# функция для восстановления матрицы
def recovery_matr(reduce_data, I, K):
  matr = np.zeros((I, K))
  for (i, k), val in reduce_data.items():
    matr[i, k] = val
  return matr

ref = np.matmul(small_matr, big_matr) # снова же воспользуемся numpy
solution = recovery_matr(reduce_data, I, K) # восстановление матрицы

np.allclose(solution, ref)

True

**Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?**

Решение не будет корректно работать, так как, если RECORDREADER-ы генерируют случайное подмножество, результат может быть неполным, поскольку некоторые элементы матриц будут отсутствовать, и произведения по этим индексам не будут рассчитаны.

**Matrix-vector multiplication #2**

In [5]:
from typing import Iterator, NamedTuple
import numpy as np

# модель элемента данных
class MatrixElement(NamedTuple):
  row: int # индекс
  col: int # столбец
  val: int # само значение

# исходя из задачи, функция для избежания полной загрузки вектора
def split_vect(st: int, size: int):
  input_vect = np.array([2, 1, 2]) # сам вектор, также пример взят из моей прошлой реализации
  return input_vect[st:st + size]

# взято из шаблона Matrix-Vector multiplication
def MAP(coordinates: (int, int), val: int):
  i, j = coordinates
  vect_portion = split_vect(j, 1) # единственное дополнение в виде загрузки элемента вектора
  yield (i, val * vect_portion[0])

def REDUCE(i: int, products: Iterator[int]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum) # (индекс строки, сумма)

def RECORDREADER():
  for i in range(mat.shape[0]): # строки
    for j in range(mat.shape[1]): # столбцы
      yield ((i, j), mat[i, j]) # произведение строки

# взятая с шаблона реализация MapReduce
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))


# взяла ту же матрицу, что и в своей первой реализации, для проверки корректности результата
# 4 * 2 + 0 * 1 + 2 * 2 = 12
# 8 * 2 + 3 * 1 + 2 * 2 = 23
# 3 * 2 + 5 * 1 + 6 * 2 = 23
matr = np.array([[4, 0, 2],
                [8, 3, 2],
                [3, 5, 6]])

output = list(MapReduce(RECORDREADER, MAP, REDUCE))
print("Matrix-Vector Multiplication:")
for key, val in output:
    print(f"Row {key}: {val}")

Matrix-Vector Multiplication:
Row 0: 12
Row 1: 23
Row 2: 23
